In [1]:
import pandas as pd
import numpy as np
import os
import pydicom
import matplotlib.pyplot as plt

from scipy.misc import imsave
from imageio import imwrite, imread
from tqdm import tqdm
from skimage import *
import itertools
import math

%matplotlib inline


import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.optimizers import SGD, Adam,Adagrad
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.layers.advanced_activations import LeakyReLU
from math import sqrt
from keras.callbacks import History 
from keras.optimizers import Adam, SGD
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,GlobalAvgPool2D, GlobalMaxPool2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense,Input,Concatenate
from keras.layers import BatchNormalization
from keras.models import Model
from keras.activations import relu
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import ELU
from keras.applications.resnet50 import preprocess_input, resnet50
from math import sqrt
from keras import backend as K
from keras.callbacks import History 
import gc

Using TensorFlow backend.


In [2]:
%run '/home/santhosr/Documents/Codes/utils.py'

In [3]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=50,horizontal_flip=True,validation_split=0.2)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [4]:
batchSize = 3
imageSize = (1024,1024)

In [5]:
trainGen = train_datagen.flow_from_directory('/home/santhosr/Keras_Data/',batch_size=batchSize,target_size=imageSize,subset='training',seed=123)
valGen = train_datagen.flow_from_directory('/home/santhosr/Keras_Data/',batch_size=batchSize,target_size=imageSize,subset='validation',seed=123)


Found 1435 images belonging to 2 classes.
Found 357 images belonging to 2 classes.


In [12]:
def conv_layer(feature_batch, feature_map, kernel_size=(3, 3),strides=(1,1), zp_flag=False):
    if zp_flag:
        zp = ZeroPadding2D((1,1))(feature_batch)
    else:
        zp = feature_batch
    conv = Conv2D(filters=feature_map, kernel_size=kernel_size, strides=strides)(zp)
#     bn = BatchNormalization(axis=3)(conv)
#     act = LeakyReLU(1/10)(bn)
    act = LeakyReLU(1/10)(conv)
    return act

In [23]:
inp = Input(shape=(imageSize[0], imageSize[1],3))

conv1 = conv_layer(inp, 64, kernel_size=(3,3), strides=(1,1),zp_flag=False)
conv2 = conv_layer(conv1, 64, kernel_size=(3,3), strides=(1,1), zp_flag=False)
mp1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv2)
# 23
conv3 = conv_layer(mp1, 128, zp_flag=False)
conv4 = conv_layer(conv3, 128, zp_flag=False)
conv4_2 = conv_layer(conv4, 128, zp_flag=False)
mp2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv4)
9
conv7 = conv_layer(mp2, 256, zp_flag=False)
conv8 = conv_layer(conv7, 256, zp_flag=False)
conv9 = conv_layer(conv8, 256, zp_flag=False)
mp3 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv9)

conv10 = conv_layer(mp3, 256, zp_flag=False)
conv11 = conv_layer(conv10, 256, zp_flag=False)
mp4 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv11)

conv12 = conv_layer(mp4, 256, strides=(2,2),zp_flag=False)
# conv13 = conv_layer(conv12, 256, zp_flag=False)
# mp5 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv13)



# dense layers
# flt = Flatten()(conv12)
out1 = GlobalAvgPool2D()(conv12)
out2 = GlobalMaxPool2D()(conv12)
out = concatenate([out1,out2])
ds1 = Dense(256, activation='relu')(out)
dp1 = Dropout(0.4)(ds1)
# ds2 = Dense(64, activation='relu')(ds1)
out = Dense(2, activation='softmax')(dp1)

model = Model(inp, out)

In [21]:
e = model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 510, 510, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_12 (LeakyReLU)      (None, 510, 510, 64) 0           conv2d_12[0][0]                  
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 508, 508, 64) 36928       leaky_re_lu_12[0][0]             
__________________________________________________________________________________________________
leaky_re_l

In [24]:
model.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics=['accuracy',f1])

In [25]:
def get_model_memory_usage(batch_size, model):
    import numpy as np
    from keras import backend as K

    shapes_mem_count = 0
    for l in model.layers:
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    number_size = 4.0
    if K.floatx() == 'float16':
         number_size = 2.0
    if K.floatx() == 'float64':
         number_size = 8.0

    total_memory = number_size*(batch_size*shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3)
    return gbytes

In [26]:
get_model_memory_usage(15, model)

2.582

In [26]:
epochs = 3

In [27]:
classWt = {0:1,1:20}

In [29]:
model.fit_generator(train_flow,epochs=epochs,verbose=1,steps_per_epoch=1500,class_weight=classWt,validation_data=valGen,validation_steps=200)

Epoch 1/3
1500/1500 [==============================] - 1094s 730ms/step - loss: 2.1746 - acc: 0.2009 - f1: 0.2009 - val_loss: 1.4793 - val_acc: 0.1956 - val_f1: 0.1956
Epoch 2/3
1500/1500 [==============================] - 1086s 724ms/step - loss: 2.1702 - acc: 0.1998 - f1: 0.1998 - val_loss: 1.5532 - val_acc: 0.1939 - val_f1: 0.1939
Epoch 3/3
1500/1500 [==============================] - 1100s 733ms/step - loss: 2.1631 - acc: 0.1981 - f1: 0.1981 - val_loss: 1.4085 - val_acc: 0.1940 - val_f1: 0.1940


In [23]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

### Transfer Learning

In [6]:
from keras.applications import ResNet50

In [27]:
model = ResNet50(include_top=False, input_shape=(1024,1024,3))

In [28]:
for layer in model.layers:
    layer.trainable = True

In [29]:
x = model.output

out1 = GlobalMaxPooling2D()(x)
out2 = GlobalAveragePooling2D()(x)
out3 = Flatten()(x)
out = Concatenate(axis=-1)([out1, out2])
out = Dropout(0.4)(out)
out = Dense(256, activation="relu")(out)
predictions = Dense(2, activation="softmax")(out)

In [30]:
fullModel = Model(inputs=model.input, outputs=predictions)

adam = Adam(lr=0.00001)

fullModel.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])

fullHistory = []

In [31]:
fullModel.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1024, 1024, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 1030, 1030, 3 0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 512, 512, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 512, 512, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [32]:
checkPoint = ModelCheckpoint(filepath='./model_resnet_{epoch:02d}_{val_loss:.4f}_{val_acc:4f}.hdf5',verbose=1,save_best_only=False,mode=min)

def step_decay(epoch):
    
    initial_lrate = 0.00001
    drop = 0.4
    epochs_drop = 2.0
    lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
    return lrate

lrate = LearningRateScheduler(step_decay)

In [33]:
fullModel.layers[-8].trainable

True

In [ ]:
full

In [22]:
history = fullModel.fit_generator(trainGen, epochs = 15, steps_per_epoch=300,validation_data=valGen,validation_steps=70,callbacks=[checkPoint])
fullHistory.append(history)

Epoch 1/15
300/300 [==============================] - 225s 749ms/step - loss: 4.7769 - acc: 0.5256 - val_loss: 2.9299 - val_acc: 0.4762

Epoch 00001: saving model to ./model_resnet_01_2.9299_0.476190.hdf5
Epoch 2/15
300/300 [==============================] - 212s 707ms/step - loss: 4.6028 - acc: 0.5289 - val_loss: 1.9434 - val_acc: 0.5000

Epoch 00002: saving model to ./model_resnet_02_1.9434_0.500000.hdf5
Epoch 3/15
300/300 [==============================] - 214s 714ms/step - loss: 4.4854 - acc: 0.5267 - val_loss: 3.8043 - val_acc: 0.5000

Epoch 00003: saving model to ./model_resnet_03_3.8043_0.500000.hdf5
Epoch 4/15
300/300 [==============================] - 216s 722ms/step - loss: 4.0805 - acc: 0.5211 - val_loss: 2.3899 - val_acc: 0.5143

Epoch 00004: saving model to ./model_resnet_04_2.3899_0.514286.hdf5
Epoch 5/15
300/300 [==============================] - 215s 717ms/step - loss: 3.7709 - acc: 0.5344 - val_loss: 1.9487 - val_acc: 0.5143

Epoch 00005: saving model to ./model_resnet

In [ ]:

for i in range(3):
    history = fullModel.fit_generator(trainGen,epochs=12,steps_per_epoch=300,validation_data=valGen, validation_steps=80,callbacks=[lrate,checkPoint])
    fullHistory.append(history)

Epoch 1/12
300/300 [==============================] - 238s 794ms/step - loss: 5.5016 - acc: 0.4856 - val_loss: 2.2911 - val_acc: 0.4833

Epoch 00001: saving model to ./model_resnet_01_2.2911_0.483333.hdf5
Epoch 2/12
300/300 [==============================] - 223s 743ms/step - loss: 4.8744 - acc: 0.4844 - val_loss: 2.1547 - val_acc: 0.5083

Epoch 00002: saving model to ./model_resnet_02_2.1547_0.508333.hdf5
Epoch 3/12
300/300 [==============================] - 226s 752ms/step - loss: 4.6379 - acc: 0.4789 - val_loss: 1.7124 - val_acc: 0.5458

Epoch 00003: saving model to ./model_resnet_03_1.7124_0.545833.hdf5
Epoch 4/12
300/300 [==============================] - 224s 746ms/step - loss: 4.3852 - acc: 0.5033 - val_loss: 2.0444 - val_acc: 0.5833

Epoch 00004: saving model to ./model_resnet_04_2.0444_0.583333.hdf5
Epoch 5/12
300/300 [==============================] - 224s 746ms/step - loss: 3.9857 - acc: 0.5356 - val_loss: 1.8655 - val_acc: 0.5417

Epoch 00005: saving model to ./model_resnet

### Testing

In [22]:
testFolder0='/home/santhosr/cbica/home/Datasets/jpeg/0'
testFolder1='/home/santhosr/cbica/home/Datasets/jpeg/1'

In [24]:
folderList = os.listdir(testFolder0)

predList = []

for img in folderList:
    
    img = read_img(os.path.join(testFolder0,img),imageSize,False)
    pred = model.predict(img.reshape(1,imageSize[0],imageSize[1],3))
    predList.append(pred)
    
    

In [25]:
predList

[array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], dtype=float32),
 array([[1., 0.]], d

In [26]:
len(os.listdir(testFolder0))

914

In [27]:
len(os.listdir(testFolder1))

228

In [29]:
228/float(228+914)

0.19964973730297722